체질량지수(BMI)
- 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈 값

# 1.data load

In [ ]:
# BMI 예제
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler       # Normalization
from sklearn.model_selection import train_test_split # train, test 분리
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Raw Data Loading
df = pd.read_csv('/content/bmi.csv', skiprows=3)
print(df.head(), df.shape)

##### 결측치 확인 #####
# df.isnull().sum()   # 결측치 없음. 만약 있으면 결측치 처리

##### 이상치 확인 #####
# zscore=1.8
# df.loc[np.abs(stats.zscore(df['height'])) >= zscore,:]   # height의 이상치 확인
# df.loc[np.abs(stats.zscore(df['weight'])) >= zscore,:]   # weight의 이상치 확인
# df.loc[np.abs(stats.zscore(df['label'])) >= zscore,:]    # label의 이상치 확인
##### 이상치 없음. 만약 있으면 이상치 처리 #####

x_data = df[['height','weight']].values
t_data = df['label'].values.reshape(-1, 1)

# Min-Max Normalization
scaler = MinMaxScaler()   # scaler = StandardScaler()
scaler.fit(x_data)
x_data_norm = scaler.transform(x_data)

# 2. data split

In [ ]:
# Data Split
x_data_train_norm, x_data_test_norm, t_data_train, t_data_test = \
train_test_split(x_data_norm,
                 t_data,
                 test_size=0.3,
                 stratify=t_data,
                 random_state=0)

# PyTorch용 Tensor로 변환
x_train_tensor = torch.FloatTensor(x_data_train_norm)
x_test_tensor = torch.FloatTensor(x_data_test_norm)
y_train_tensor = torch.LongTensor(t_data_train.flatten())
y_test_tensor = torch.LongTensor(t_data_test.flatten())

# Dataset과 DataLoader 생성
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 3.model 정의

In [ ]:
# 모델 정의
class LogisticRegressionModel(nn.Module):
    def __init__(self):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(2, 3)  # input_dim=2, output_dim=3 (softmax class)

    def forward(self, x):
        return self.linear(x)

model = LogisticRegressionModel()

# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()  # 'sparse_categorical_crossentropy'에 해당
optimizer = optim.SGD(model.parameters(), lr=1e-2)

# 4.학습

In [ ]:
# 학습 루프
epochs = 200
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# 5.평가

In [ ]:
# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)

        """
        print(f'labels.shape = {labels.shape}')   # labels.shape = torch.Size([32])
        print(f'outputs.shape = {outputs.shape}') # outputs.shape = torch.Size([32, 3])
        print(f'labels = {labels}')               # labels = tensor([1, 2, 0, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1])
        print(f'outputs[0] = {outputs[0]}')       # outputs[0] = tensor([-1.7481,  0.5193,  0.5558])
        print(f'predicted = {predicted}')         # predicted = tensor([2, 2, 0, 2, 0, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 0, 0, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1])
        """

        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy:.4f}')
# Test Accuracy: 0.9822

# 6.예측

In [ ]:
# 예측
height = 187
weight = 83
my_state = [[height, weight]]
my_state_val = scaler.transform(my_state)
my_state_tensor = torch.FloatTensor(my_state_val)

model.eval()
with torch.no_grad():
    predict_result = model(my_state_tensor) # predict_result = tensor([[-3.2538,  0.7598,  0.1596]])
    print(f'predict_result = {predict_result}')
    predict_prob = torch.softmax(predict_result, dim=1)
    print(f'predict_prob = {predict_prob}') # predict_prob = tensor([[0.0115, 0.6383, 0.3502]])

# 예측 출력
# 예시 출력은 다음과 같이 Softmax 값으로 나옵니다.
# tensor([[0.0116, 0.6356, 0.3528]])